In [1]:
import pandas as pd
import json
from datetime import datetime
import os
import re

In [2]:
class my_dictionary(dict): 
  
    # __init__ function 
    def __init__(self): 
        self = dict() 
          
    # Function to add key:value 
    def add(self, key, value): 
        self[key] = value 


# 1- cleaning data

In [3]:
df = pd.read_csv('khakha.csv')
dfwin = pd.read_csv('../AwardDates.csv')
data = dict()
with open('../dates.json') as json_file:
    data = json.load(json_file)
for d in data: 
    date = d['date']
    result = re.search('Published(.*)', date)
    partial = result.group(1)
    if('by' in partial) :
        partial = re.search('(.*)by', partial)
        d['date'] = partial.group(1).lstrip().rstrip()
    else:
        d['date'] = partial.lstrip().rstrip()

with open('new.json', 'w') as new:
    json.dump(data, new)

In [4]:
# calculating dates and matching isbn
data = dict()
with open('new.json') as json_file:
    data = json.load(json_file)
hi = my_dictionary()
h2 = my_dictionary()
isbns = list()
for d in data:

    ds = str(d['date']).split()
    if len(ds)>1:
        ds[1]=ds[1].replace('th','').replace('st','').replace('rd','').replace('nd','')
        d['date'] = ''.join(str(e)+' ' for e in ds)
    if len(ds)>3:
         d['date'] = ''.join([ds[0],' ',ds[1],' ',ds[2],' '])
    try:
        d['date']=datetime.strptime(d['date'], '%B %d %Y ')
    except ValueError :
        try:
           d['date']= datetime.strptime(d['date'], '%B %Y ')
        except ValueError :
           d['date']= datetime.strptime(d['date'], '%Y')
    if(df[df['url']==d['url']].empty):
        continue
    year=df[df['url']==d['url']]['year'].iloc[0]
    win=datetime.strptime(dfwin[dfwin['year'] == year]['Winners Announced'].iloc[0],'%m/%d/%Y')
    if((win -  d['date']).days<1):
        d['date']=(win -  d['date']).days
    else:
        d['date']=(win -  d['date']).days 

    hi.add(d['url'], d['date'])
    h2.add(d['url'], d['isbn'])
    isbns.append(d['isbn'])

# merging data and adding 'times_appeared_NYTime','avgrank_NYTime','bestRank','listnames' to are coulems

In [5]:
dfnew = pd.DataFrame(hi.items(), columns=['url', 'daystocom'])
dfnew1 = pd.DataFrame(h2.items(), columns=['url', 'isbn'])
df.merge(dfnew, on='url', how='left').merge(dfnew1, on='url', how='left').to_csv("withdays.csv")
df = pd.read_csv('withdays.csv')
df = df.reindex(df.columns.tolist() + ['times_appeared_NYTime','avgrank_NYTime','bestRank','numWeeks', 'listnames'], axis=1)

In [6]:
def set_default_values():
    values = {}
    values['bestrank'] = float('inf')
    values['sum_ranks'] =  float('inf')
    values['count'] = 0
    values['listnames'] = set()
    values['weeks'] = dict()
    return values

def set_row(myDf, values, isbn, year, key):
    myDf.loc[(myDf[key] == isbn) & (myDf['year'] == year), 'times_appeared_NYTime'] = values['count']
    if((values['count'] == 0) or (values['sum_ranks'] == float('inf'))):
        avg = float('inf')
    else:
        avg =  values['sum_ranks'] / values['count']
    myDf.loc[(myDf[key]==isbn) & (myDf['year'] == year), 'avgrank_NYTime'] = avg
    myDf.loc[(myDf[key]==isbn) & (myDf['year'] == year), 'bestRank'] = values['bestrank']
    myDf.loc[(myDf[key]==isbn) & (myDf['year'] == year), 'listnames'] = len(values['listnames'])
    myDf.loc[(myDf[key]==isbn) & (myDf['year'] == year), 'numWeeks'] = sum(values['weeks'].values())

def update_weeks_dict(values, listName, numWeeks):
    if(listName not in values['weeks']):
        values['weeks'][listName] = numWeeks
    else:
        values['weeks'][listName] = max(numWeeks, values['weeks'][listName])

reading newYork Data and adding to feilds

In [7]:
def add_nyt_data(data, df, isbns, key):
    for i in isbns:
        # will be handled later
        if i =='English': continue         
        # for each year, we need to calculate coloumn values 
        years = df[df[key] == i]['year']
        for year in years:
            values = {'bestrank':float('inf'), 'sum_ranks':0, 'count':0, 'listnames' : set(), 'weeks' : dict()}
            # didn't appear in nyt list   
            if len(data[i]['results'])==0:    
                values = set_default_values()
                set_row(df, values, i, year, key)
                continue

            win = datetime.strptime(dfwin[dfwin['year'] == year]['Winners Announced'].iloc[0],'%m/%d/%Y')
            
            for z in data[i]['results'][0]['ranks_history']:
                if(datetime.strptime(z['published_date'], '%Y-%m-%d') < win):
                    values['sum_ranks'] += z['rank']
                    values['count'] = values['count'] + 1
                    values['listnames'].add(z['list_name'])
                    update_weeks_dict(values, z['list_name'], z['weeks_on_list'])
                    if values['bestrank'] > z['rank']:
                        values['bestrank'] = z['rank']
                        
            if values['count'] > 0:
                set_row(df, values, i, year, key)
            else:
                values = set_default_values()
                set_row(df, values, i, year, key)

In [8]:
data = dict()
newYork_dir = "../newYork Data/"
data = dict()
new_data = dict()
for filename in os.listdir(newYork_dir):
    if (filename.endswith(".json") and filename.startswith("res")): 
        cmplt_path = os.path.join(newYork_dir, filename)
        print(cmplt_path)
        with open(cmplt_path) as json_file :
            new_data = json.load(json_file)
            data.update(new_data)
    else:
        continue

print(len(isbns))
key = 'isbn'
add_nyt_data(data, df, isbns, key)

../newYork Data/res8.json
../newYork Data/res4.json
../newYork Data/res5.json
../newYork Data/res10.json
../newYork Data/res9.json
../newYork Data/res2.json
../newYork Data/res3.json
../newYork Data/res1.json
../newYork Data/res6.json
../newYork Data/res7.json
3969
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update

update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update

update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update

update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update

update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update

update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update

update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update

update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update

saving data

In [9]:
df.to_csv("partial.csv")

cheak nall values

In [10]:
df = pd.read_csv('partial.csv')
def nans(df): return df[df.isnull().any(axis=1)]
nans(df)

,Unnamed: 0,Unnamed: 0.1,average rating,average_rating_w,category,name,num 1 stars,num 2 stars,num 3 stars,num 4 stars,...,winner,writer,year,daystocom,isbn,times_appeared_NYTime,avgrank_NYTime,bestRank,numWeeks,listnames
0,0,0,3.847458,3.99,Graphic Novels & Comics,"The Umbrella Academy, Vol. 3: Hotel Oblivion",0.013559,0.077966,0.294915,0.274576,...,0,"['Gerard Way', 'Gabriel Bá']",2019,84.0,English,NaN,NaN,NaN,NaN,NaN
3,3,3,3.371930,4.20,Graphic Novels & Comics,Shades of Magic Vol. 1: The Steel Prince,0.157895,0.108772,0.242105,0.185965,...,0,"['V.E. Schwab', 'Andrea Olimpieri']",2019,279.0,English,NaN,NaN,NaN,NaN,NaN
4,4,4,3.808874,3.82,Graphic Novels & Comics,Teen Titans: Raven,0.023891,0.044369,0.279863,0.402730,...,0,"['Kami Garcia', 'Gabriel Picolo']",2019,161.0,English,NaN,NaN,NaN,NaN,NaN
7,7,7,4.367347,3.99,Graphic Novels & Comics,"""Okay""",0.000000,0.010204,0.115646,0.370748,...,0,"['Kieron Gillen', 'Jamie McKelvie', 'Matt Wils...",2019,63.0,English,NaN,NaN,NaN,NaN,NaN
10,10,10,4.305085,4.20,Graphic Novels & Comics,"Fence, Vol. 3",0.003390,0.013559,0.111864,0.416949,...,0,"['C.S. Pacat', 'Johanna the Mad', 'Joana LaFue...",2019,112.0,English,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3537,3537,3537,4.372483,4.18,Fantasy,The Obelisk Gate,0.020134,0.006711,0.067114,0.392617,...,0,['N.K. Jemisin'],2016,110.0,English,NaN,NaN,NaN,NaN,NaN
3539,3539,3539,4.576667,4.28,Fantasy,The Curse of Tenth Grave,0.000000,0.003333,0.050000,0.313333,...,0,['Darynda Jones'],2016,161.0,English,NaN,NaN,NaN,NaN,NaN
3586,3586,3586,4.265993,4.19,Historical Fiction,America's First Daughter,0.010101,0.013468,0.134680,0.383838,...,0,"['Stephanie Dray', 'Laura Kamoie']",2016,280.0,English,NaN,NaN,NaN,NaN,NaN
3611,3611,3611,4.070234,3.90,Science & Technology,The Inevitable: Understanding the 12 Technolog...,0.026756,0.030100,0.173913,0.384615,...,0,['Kevin Kelly'],2016,182.0,English,NaN,NaN,NaN,NaN,NaN


read data from json 11 with different struct and saving to csv

In [12]:
key = 'url'
with open('../newYork Data/remained.json') as json_file:
    data = json.load(json_file)
    for i in data:        
        years = df[df[key] == i]['year']
        for year in years:
            # didn't appear in any list
            if len(data[i]['results']) == 0:
                values = set_default_values()
                set_row(df, values, i, year, key)
                continue

            values = {'bestrank':float('inf'), 'sum_ranks':0, 'count':0, 'listnames' : set(), 'weeks' : dict()}
            win=datetime.strptime(dfwin[dfwin['year'] == year]['Winners Announced'].iloc[0],'%m/%d/%Y')

            for z in data[i]['results'][0]['ranks_history']:
                if(datetime.strptime(z['published_date'], '%Y-%m-%d')<win):
                    values['sum_ranks'] += z['rank']
                    values['count'] = values['count'] + 1
                    values['listnames'].add(z['list_name'])
                    update_weeks_dict(values, z['list_name'], z['weeks_on_list'])
                    if values['bestrank'] > z['rank']:
                        values['bestrank'] = z['rank']
            if values['count'] > 0:
                set_row(df, values, i, year, key)
            else:
                values = set_default_values()
                set_row(df, values, i, year, key)
df.to_csv("final.csv")

update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update

saving nulls in file

In [ ]:
def nans(df): return df[df.isnull().any(axis=1)]
nans(df).to_csv("nulls.csv")

In [ ]:
# dfnew=pd.DataFrame(hi.items(), columns=['url', 'daystocom'])
# dfnew1 = pd.DataFrame(h2.items(), columns=['url', 'isbn'])
# # dfnew2 = pd.DataFrame(h5.items(), columns=['isbn', 'times_appred_NYTime'])
# # dfnew3 = pd.DataFrame(h4.items(), columns=['isbn', 'avgrank_NYTime'])
# # dfnew4 = pd.DataFrame(h6.items(), columns=['isbn', 'bestRank'])
# # dfnew5 = pd.DataFrame(h7.items(), columns=['isbn', 'listnames'])
# df.merge(dfnew,on='url', how='left').merge(dfnew1,on='url', how='left').to_csv("withdays.csv")
# # .merge(dfnew3,on='isbn', how='left').merge(dfnew4,on='isbn', how='left').merge(dfnew5,on='isbn', how='left').to_csv("withdays.csv")

In [ ]:
df = pd.read_csv('../goodreads_output3.csv')
df

In [ ]:
df1 = pd.read_csv('../test.csv')
new = df1[df1['year']== 2019].drop(['daysToContest', 'isbn', 'times_appred_NYTime', 'avgrank_NYTime', 'bestRank', 'listnames'], axis = 1)
def add(a, b, c, d, e): 
    return a + b + c
new[['num 1 stars', 'num 2 stars', 'num 3 stars', 'num 4 stars', 'num 5 stars']] = new[['num 1 stars', 'num 2 stars', 'num 3 stars', 'num 4 stars', 'num 5 stars']].div(new.iloc[:, -10:-5].sum(axis=1), axis=0)
new

In [ ]:
before = pd.read_csv('../goodreads_output3.csv')
df.columns
output =  new.append(before) 
ourput = output.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'], axis = 1)

In [ ]:
 ourput

In [ ]:
ourput.to_csv('khakha.csv', index=False)

In [ ]:
#df = pd.read_csv('final.csv')
#df[df['avgrank_NYTime'] < 100]

In [ ]:
#df

In [ ]:
df[['name', 'category']]

In [ ]:
dict = {}
print(sum(dict.values()))